# LeanTrader Backtest Demo

This notebook demonstrates how to backtest strategies for **XAUUSD, EURUSD, BTCUSDT** using the LeanTrader framework.

Steps:
1. Load OHLC data (CSV or feeds)
2. Resample to required timeframes
3. Run features → DSL strategy → signals
4. Backtest with ATR-based SL/TP
5. Plot equity curve and metrics


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from leantrader.data.feeds import get_ohlc_csv, resample_frames
from leantrader.features.microstructure import engineer
from leantrader.policy.dispatcher import run_for_pair
from leantrader.backtest.engine import backtest
from leantrader.backtest.metrics import sharpe, max_drawdown
from leantrader.config import CFG
import os


In [ ]:
pairs = ["XAUUSD", "EURUSD", "BTCUSDT"]
results = {}

for pair in pairs:
    path = f"data/ohlc/{pair}_M15.csv"
    if not os.path.exists(path):
        print(f"Missing {path}, skip.")
        continue
    df = get_ohlc_csv(path)
    frames = resample_frames(df)
    eng_frames = {k: engineer(v) for k,v in frames.items()}
    sigs = run_for_pair(pair, eng_frames)
    eq = backtest(eng_frames["M15"], sigs, CFG.risk)
    results[pair] = (sigs, eq)
    
    # Plot equity
    eq.plot(title=f"Equity Curve: {pair}")
    plt.show()
    
    print(pair, "Sharpe:", sharpe(eq), "MaxDD:", max_drawdown(eq))
